In [2]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))
from cnv import utilities as cnv_util
from cnv import coverage_matrix as cm
import pandas as pd
from genepeeks.common import utilities as util
import tabletext

INFO:config:Using default ini file from: /Users/msilver/Dropbox (GenePeeks)/Max_workspace/genepeeks-science/genepeeks/common/../../conf/genepeeks.ini


In [83]:
reload(cnv_util)
reload(util)

<module 'genepeeks.common.utilities' from '/Users/msilver/Dropbox (GenePeeks)/Max_workspace/genepeeks-science/genepeeks/common/utilities.pyc'>

#### Compile all DMD interval lists to compare (primary ensembl transcript, all ensembl transcripts, or from TruSight Inherited Disease). Can optionally choose to restrict to only coding regions, or to include full exons including UTR

In [87]:
def get_targets_d(only_coding_regions):
    interval_args = {'min_dist': 200, 'only_coding_regions': only_coding_regions, 'print_merged_exons': False}
    targets_d = {
        'full_ensembl': {'intervals': cnv_util.get_merged_exons(only_primary_transcript=False, **interval_args)[0]},
        'primary': {'intervals': cnv_util.get_merged_exons(only_primary_transcript=True, **interval_args)[0]},
    }
    bedfile_paths = {
        'TSID': os.path.join('..', 'inputs', 'TruSight_Inherited_Disease_Manifest_A.bed'),
    #     'TSO': os.path.join('..', 'inputs', 'TruSight-One-BED-May-2014.txt')
    }
    for name, file_p in bedfile_paths.items():
        df = pd.read_csv(file_p, delimiter='\t', header=None, names=('chrom', 'start', 'end', 'id'))
        X_df = df[df['chrom'] == 'chrX'].sort_values(by='start')
        X_df['gene'] = [ID.split('.')[0] for ID in X_df['id']]
        DMD_intervals = map(dict, dict(X_df[X_df['gene'] == 'DMD'].T).values())
        targets_d[name] = {'intervals': util.merge_intervals(DMD_intervals, include_index=True, min_dist=200)}
    return targets_d

#### Output all intervals that vary between the different panel's intervals

In [79]:
def output_differences(only_coding_regions):
    print 'only_coding_regions: {}'.format(only_coding_regions)
    targets_d = get_targets_d(only_coding_regions)
    all_interval_ranges = []
    for name, targets in targets_d.items():
        targets['ranges'] = [(intrv['start'], intrv['end']) for intrv in targets['intervals']]
        all_interval_ranges.append(targets['intervals'])
    full_intervals = util.interval_union(*all_interval_ranges)

    panels = targets_d.keys()
    intrv_fields = ['Index', 'Length']
    headers = ['{}_{}'.format(panel, field) for panel in panels for field in intrv_fields]
    table = [headers]
    for intrv in full_intervals:
        region = (intrv['start'], intrv['end'])
        if any(region not in target['ranges'] for target in targets_d.values()):
            disagreement = []
            for panel in panels:
                intervals = targets_d[panel]['intervals']
                exon_indexes = util.in_interval(region, intervals)
                if exon_indexes is None:
                    index = length = start = end = None
                else:
                    target = intervals[exon_indexes[0]]
                    index = target['index']
                    start = target['start']
                    end = target['end']
                    length = target['end'] - target['start'] + 1
                disagreement += [index, length]
            table.append(disagreement)
    print tabletext.to_text(table), '\n'

#### Output differences when looking at only coding regions

In [88]:
output_differences(True)

only_coding_regions: True
┌────────────┬─────────────┬───────────────┬────────────────┬────────────────────┬─────────────────────┐
│ TSID_Index │ TSID_Length │ primary_Index │ primary_Length │ full_ensembl_Index │ full_ensembl_Length │
├────────────┼─────────────┼───────────────┼────────────────┼────────────────────┼─────────────────────┤
│ 1          │        2703 │ 1             │             12 │ 1                  │                  98 │
├────────────┼─────────────┼───────────────┼────────────────┼────────────────────┼─────────────────────┤
│ 10         │         611 │ 10            │            137 │ 10                 │                 137 │
├────────────┼─────────────┼───────────────┼────────────────┼────────────────────┼─────────────────────┤
│ 18         │          98 │ None          │ None           │ 18                 │                  20 │
├────────────┼─────────────┼───────────────┼────────────────┼────────────────────┼─────────────────────┤
│ 26         │          91 │ 

#### Output differences when including UTRs

In [84]:
output_differences(False)

only_coding_regions: False
There are 90 merged exons, with the following exons being merged: ['71|72']
There are 78 merged exons, with the following exons being merged: ['65|66']
┌────────────┬─────────────┬───────────────┬────────────────┬────────────────────┬─────────────────────┐
│ TSID_Index │ TSID_Length │ primary_Index │ primary_Length │ full_ensembl_Index │ full_ensembl_Length │
├────────────┼─────────────┼───────────────┼────────────────┼────────────────────┼─────────────────────┤
│ None       │ None        │ None          │ None           │ 1                  │                 507 │
├────────────┼─────────────┼───────────────┼────────────────┼────────────────────┼─────────────────────┤
│ 1          │        2703 │ 1             │           2703 │ 2                  │                2712 │
├────────────┼─────────────┼───────────────┼────────────────┼────────────────────┼─────────────────────┤
│ 10         │         611 │ 10            │            137 │ 11                 │    